In [2]:
import gym
import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.monitor import get_monitor_files
from optical_rl_gym.envs.rmsa_env import shortest_path_first_fit, shortest_available_path_first_fit, \
    least_loaded_path_first_fit, SimpleMatrixObservation
from optical_rl_gym.utils import evaluate_heuristic, random_policy
import pandas as pd

import pickle
import logging
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# initialization part

logging.getLogger('rmsaenv').setLevel(logging.INFO)

seed = 20
episodes = 1
episode_length =  90
incremental_traffic_percentage = 80

monitor_files = []
policies = []

# adding logging method
#log_dir = "./tmp/logrmsa-ppo/"
logging_dir = "./tmp/logrmsa-ppo-defragmentation/"
figures_floder = f'{logging_dir}/figures-{incremental_traffic_percentage}/'
os.makedirs(logging_dir, exist_ok=True)
os.makedirs(figures_floder, exist_ok=True)
min_load = 18
max_load = 60
step_length = 8
steps = int((max_load - min_load)/step_length) +1
loads = np.zeros(steps)

random_SBR_load = np.zeros(steps)
no_defrag_SBR_load = np.zeros(steps)
oldest_SBR_load = np.zeros(steps)

random_BBR_load = np.zeros(steps)
no_defrag_BBR_load = np.zeros(steps)
oldest_BBR_load = np.zeros(steps)

random_reward_load = np.zeros(steps)
no_defrag_reward_load = np.zeros(steps)
oldest_reward_load = np.zeros(steps)

random_num_move_load = np.zeros(steps)
no_defrag_num_move_load = np.zeros(steps)
oldest_num_move_load = np.zeros(steps)

random_num_defrag_load = np.zeros(steps)
no_defrag_num_defrag_load = np.zeros(steps)
oldest_num_defrag_load = np.zeros(steps)

In [4]:
for load_counter, load_traffic in enumerate(range(min_load,max_load,step_length)):
    log_dir = f'{logging_dir}logs_{load_traffic}_{episode_length}_{incremental_traffic_percentage}/'
    os.makedirs(log_dir, exist_ok=True)

    loads[load_counter] = load_traffic
    # load results
    all_results = load_results(log_dir)
    # Defining monitor files based on the below command does not work well, so i defined it statistically
    #monitor_files = get_monitor_files(log_dir)
    monitor_files = ['./tmp/logrmsa-ppo-defragmentation/df.monitor.csv',
                     './tmp/logrmsa-ppo-defragmentation/df-random.monitor.csv','./tmp/logrmsa-ppo-defragmentation/df-oldest.monitor.csv']
    SBR = dict()
    BBR= dict()
    reward= dict()
    num_move= dict()
    num_defrag = dict()
    counter = 0

    for file_names in monitor_files:
        SBR [file_names] = all_results.loc[counter:counter+ episodes-1,'episode_service_blocking_rate'].to_list()
        BBR[file_names] = all_results.loc[counter:counter + episodes-1, 'episode_bit_rate_blocking_rate'].to_list()
        reward[file_names] = all_results.loc[counter:counter + episodes-1, 'reward'].to_list()
        num_move[file_names] = all_results.loc[counter:counter + episodes-1, 'number_movements'].to_list()
        num_defrag[file_names] = all_results.loc[counter:counter + episodes-1, 'number_defragmentation_procedure'].to_list()
        counter = counter + episodes
    
    for key, value in SBR.items():
        if 'df-oldest' in key:
            oldest_SBR = value
        elif 'df-random' in key:
            random_SBR = value
        else:
            no_defrag_SBR = value

    for key, value in BBR.items():
        if 'df-oldest' in key:
            oldest_BBR = value
        elif 'df-random' in key:
            random_BBR = value
        else:
            no_defrag_BBR = value
    
    
    for key, value in reward.items():
        if 'df-oldest' in key:
            oldest_reward = value
        elif 'df-random' in key:
            random_reward = value
        else:
            no_defrag_reward = value

    for key, value in num_move.items():
        if 'df-oldest' in key:
            oldest_num_move = value
        elif 'df-random' in key:
            random_num_move = value
        else:
            no_defrag_num_move = value    
            
    for key, value in num_defrag.items():
        if 'df-oldest' in key:
            oldest_num_defrag = value
        elif 'df-random' in key:
            random_num_defrag = value
        else:
            no_defrag_num_defrag = value    
    
    random_SBR_load[load_counter] = np.mean(random_SBR)
    oldest_SBR_load[load_counter] = np.mean(oldest_SBR)
    no_defrag_SBR_load[load_counter] = np.mean(no_defrag_SBR)
    
    random_BBR_load[load_counter] = np.mean(random_BBR)
    oldest_BBR_load[load_counter] = np.mean(oldest_BBR)
    no_defrag_BBR_load[load_counter] = np.mean(no_defrag_BBR)
    
    random_reward_load[load_counter] = np.mean(random_reward)
    oldest_reward_load[load_counter] = np.mean(oldest_reward)
    no_defrag_reward_load[load_counter] = np.mean(no_defrag_reward)
    
    random_num_defrag_load[load_counter] = np.mean(random_num_defrag)
    oldest_num_defrag_load[load_counter] = np.mean(oldest_num_defrag)
    no_defrag_num_defrag_load[load_counter] = np.mean(no_defrag_num_defrag)
    
    random_num_move_load[load_counter] = np.mean(random_num_move)
    oldest_num_move_load[load_counter] = np.mean(oldest_num_move)
    no_defrag_num_move_load[load_counter] = np.mean(no_defrag_num_move)
    
    
    

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in ./tmp/logrmsa-ppo-defragmentation/logs_18_90_80/

In [ ]:
## plotting the BBR

fig = plt.figure(figsize=[8.4, 4.8])
plt.semilogy(loads, random_BBR_load, '+-r', label = 'random_BBR')
plt.semilogy(loads, oldest_BBR_load, '+-k', label = 'oldest_BBR')
plt.semilogy(loads, no_defrag_BBR_load, '+-y', label = 'no_defrag_BBR')
plt.xlabel('load')
plt.ylabel('Bit Blocking Rate')
plt.legend()
plt.savefig(f'{figures_floder}/bit_blocking.pdf')
plt.savefig(f'{figures_floder}/bit_blocking.svg')
plt.show()
plt.close()


In [ ]:
## plotting the SBR

fig = plt.figure(figsize=[8.4, 4.8])
plt.semilogy(loads, random_SBR_load, '+-r', label = 'random_SBR')
plt.semilogy(loads, oldest_SBR_load, '+-k', label = 'oldest_SBR')
plt.semilogy(loads, no_defrag_SBR_load, '+-y', label = 'no_defrag_SBR')
plt.xlabel('load')
plt.ylabel('Service Blocking Rate')
plt.legend()
plt.savefig(f'{figures_floder}/service_blocking.pdf')
plt.savefig(f'{figures_floder}/service_blocking.svg')
plt.show()
plt.close()

In [ ]:
## plotting reward

fig = plt.figure(figsize=[8.4, 4.8])
plt.plot(loads, random_reward_load, '+-r', label = 'random_reward')
plt.plot(loads, oldest_reward_load, '+-k', label = 'oldest_reward')
plt.plot(loads, no_defrag_reward_load, '+-y', label = 'no_defrag_reward')
plt.xlabel('load')
plt.ylabel('reward')
plt.legend()
plt.savefig(f'{figures_floder}/reward.pdf')
plt.savefig(f'{figures_floder}/reward.svg')
plt.show()
plt.close()

In [ ]:
## plotting number of movements

fig = plt.figure(figsize=[8.4, 4.8])
plt.plot(loads, random_num_move_load, '+-r', label = 'random_num_move')
plt.plot(loads, oldest_num_move_load, '+-k', label = 'oldest_num_move')
plt.plot(loads, no_defrag_num_move_load, '+-y', label = 'no_defrag_num_move')
plt.xlabel('load')
plt.ylabel('number of movements')
plt.legend()
plt.savefig(f'{figures_floder}/num_move.pdf')
plt.savefig(f'{figures_floder}/num_move.svg')
plt.show()
plt.close()

In [ ]:
## plotting number of defragmentation procedure

fig = plt.figure(figsize=[8.4, 4.8])
plt.plot(loads, random_num_defrag_load, '+-r', label = 'random_num_defrag')
plt.plot(loads, oldest_num_defrag_load, '+-k', label = 'oldest_num_defrag')
plt.plot(loads, no_defrag_num_defrag_load, '+-y', label = 'no_defrag_num_defrag')
plt.xlabel('load')
plt.ylabel('number of defragmentation')
plt.legend()
plt.savefig(f'{figures_floder}/num_defrag.pdf')
plt.savefig(f'{figures_floder}/num_defrag.svg')
plt.show()
plt.close()